In [2]:
from seleniumbase import SB
from PIL import Image
import io
import re
from selenium.webdriver.common.by import By

In [3]:
anos = [2025, 2024, 2023]

In [ ]:
//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]
//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]
//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]

In [ ]:
<div class="mdDown:bg_surface.s1 mdDown:br_lg mdDown:elevation_2 mdDown:pos_relative md:bg_surface.s1 md:br_lg md:elevation_2 md:pos_relative card-component undefined">

In [4]:
from seleniumbase import SB

xpath_competicao = '//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div[1]/button/div'
xpath_ano = '//*[@id="__next"]/main/div[2]/div/div/div[1]/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[2]/div/div/button/div'
xpath_todos_jogadores = '//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]/div[2]/div/div[1]/div[2]/button/div'
xpath_mostrar_mais = '//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]/div[3]/div/button'
xpath_temporada = '//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div[2]/button'

anos = [2025, 2024, 2023]
jogadores_por_ano = {}  # {ano: {time: [(nome, id)]}}
times_por_ano = {}      # {ano: [links_times]}
dados_temporadas = []

try:
    with SB(uc=True) as sb:
        sb.open('https://www.sofascore.com/pt/')
        sb.sleep(2)

        sb.click('div[data-tabid="competitions"][data-testid="tab-competitions"]')
        sb.sleep(2)
        sb.click('span:contains("Brasil")')
        sb.sleep(2)
        sb.click('span:contains("Brasileirão Betano")')
        sb.sleep(3)

        # --- ETAPA 1: Coleta de links dos times por ano ---
        for ano in anos:
            print(f"\n🔹 Coletando times para ano: {ano}")
            sb.click(xpath_ano)
            sb.sleep(1)
            sb.click(f'ul.Box.klGMtt li:contains("{ano}")')
            sb.sleep(2)
            
            url = sb.get_current_url()
            parts = url.split("/")
            unique_tournament_id = parts[-1].split("#")[0]
            season_id = url.split("#id:")[-1]
            
            dados_temporadas.append({
            "ano": ano,
            "unique_tournament": unique_tournament_id,
            "season": season_id
            })

            hrefs_times = sb.find_elements('div[data-panelid="all"].TabPanel.bpHovE a')
            links_times = [a.get_attribute("href") for a in hrefs_times if a.get_attribute("href")]

            times_por_ano[ano] = links_times
            print(f"➡️ {len(links_times)} times encontrados para {ano}")

        print(dados_temporadas)
        # --- ETAPA 2: Iteração nos times ---
        for ano, links_times in times_por_ano.items():
            print(f"\n🔹 Processando jogadores do ano: {ano}")
            jogadores_por_ano[ano] = {}

            for link_time in links_times:
                time_nome = link_time.split('/')[6]
                print(f"\n➡️ Processando time: {time_nome}")

                sb.open(link_time)
                sb.sleep(0.5)

                # Scroll até o botão de competição aparecer
                max_scrolls = 20
                scroll_count = 0
                while not sb.is_element_visible(xpath_competicao) and scroll_count < max_scrolls:
                    sb.execute_script("window.scrollBy(0, 300);")
                    sb.sleep(0.5)
                    scroll_count += 1

                if sb.is_element_visible(xpath_competicao):
                    sb.scroll_to(xpath_competicao)
                    sb.sleep(0.5)
                    sb.click(xpath_competicao)
                    sb.sleep(0.5)
                    sb.click('ul.Box.klGMtt li:contains("Brasileirão Betano")')
                    sb.click(xpath_temporada)
                    sb.sleep(0.5)
                    sb.click(f'ul.Box.klGMtt li:contains("{ano}")')
                    sb.click(xpath_todos_jogadores)
                    sb.sleep(0.5)
                    sb.click('ul.Box.klGMtt li:contains("Todos os jogadores")')
                    sb.sleep(0.5)

                    # Clica em "Mostrar mais" se existir
                    if sb.is_element_visible(xpath_mostrar_mais):
                        sb.click(xpath_mostrar_mais)
                        sb.sleep(1)

                    # Coleta de jogadores via XPath
                    hrefs_jogadores = sb.find_elements(
                        '//*[@id="__next"]/main/div[2]/div/div/div[1]/div[5]/div[1]/div[4]//a'
                    )
                    jogadores_tuplas = []
                    for a in hrefs_jogadores:
                        link = a.get_attribute("href")
                        if link and "/player/" in link:
                            parts = link.split("/")
                            nome = parts[-2]
                            jid = parts[-1]
                            jogadores_tuplas.append((nome, jid))
                            print(f"   Jogador: {nome} | ID: {jid}")

                    jogadores_por_ano[ano][time_nome] = jogadores_tuplas
                    print(f"✅ Total de jogadores armazenados para {time_nome}: {len(jogadores_tuplas)}")
                else:
                    print(f"⚠ Elemento de competição não encontrado para o time {time_nome}.")

except Exception as e:
    print(f"❌ Erro: {e}")

# Resultado final
print(jogadores_por_ano)



🔹 Coletando times para ano: 2025
➡️ 20 times encontrados para 2025

🔹 Coletando times para ano: 2024
➡️ 20 times encontrados para 2024

🔹 Coletando times para ano: 2023
➡️ 20 times encontrados para 2023
[{'ano': 2025, 'unique_tournament': '325', 'season': '72034'}, {'ano': 2024, 'unique_tournament': '325', 'season': '58766'}, {'ano': 2023, 'unique_tournament': '325', 'season': '48982'}]

🔹 Processando jogadores do ano: 2025

➡️ Processando time: flamengo
   Jogador: giorgian-de-arrascaeta | ID: 333587
   Jogador: wesley-franca | ID: 1134200
   Jogador: leo-ortiz | ID: 869643
   Jogador: gerson | ID: 611876
   Jogador: agustin-rossi | ID: 1140986
   Jogador: leo-pereira | ID: 358548
   Jogador: samuel-lino | ID: 874705
   Jogador: luiz-araujo | ID: 840451
   Jogador: ayrton-lucas | ID: 818473
   Jogador: erick-pulgar | ID: 590262
   Jogador: alex-sandro | ID: 84854
   Jogador: gonzalo-plata | ID: 937937
   Jogador: jorginho | ID: 132874
   Jogador: danilo | ID: 124992
   Jogador: pedro

In [ ]:
import json
from seleniumbase import SB

heatmaps_por_jogador = {}

with SB(uc=True) as sb:
    for ano in anos:
        temporada = next((t for t in dados_temporadas if t['ano'] == ano), None)

        if temporada and ano in jogadores_por_ano:
            id_unique_tournament = temporada['unique_tournament']
            id_season = temporada['season']
            heatmaps_por_jogador[ano] = {}

            for time, jogadores in jogadores_por_ano[ano].items():
                heatmaps_por_jogador[ano][time] = {}

                for nome, id_jogador in jogadores:
                    # URL da página que contém o JSON dentro de <pre>
                    url = f"https://www.sofascore.com/api/v1/player/{id_jogador}/unique-tournament/{id_unique_tournament}/season/{id_season}/heatmap/overall"
                    try:
                        sb.open(url)  # abre a página
                        # Pega o texto do <pre>
                        pre_text = sb.get_text("/html/body/pre")
                        # Converte em dicionário
                        data = json.loads(pre_text)
                        
                        # Extrai os pontos
                        pontos = data.get("points", [])
                        heatmaps_por_jogador[ano][time][nome] = pontos
                        
                        print(f"✅ Ano {ano} | {time} | {nome} -> {len(pontos)} pontos")
                    except Exception as e:
                        print(f"⚠️ Erro ao pegar heatmap de {nome}: {e}")

# Exemplo: visualizar parte dos dados
print(json.dumps(heatmaps_por_jogador, indent=2)[:500])

✅ Ano 2025 | flamengo | giorgian-de-arrascaeta -> 868 pontos
✅ Ano 2025 | flamengo | wesley-franca -> 773 pontos
✅ Ano 2025 | flamengo | leo-ortiz -> 1354 pontos
✅ Ano 2025 | flamengo | gerson -> 568 pontos
✅ Ano 2025 | flamengo | agustin-rossi -> 445 pontos
✅ Ano 2025 | flamengo | leo-pereira -> 1333 pontos
✅ Ano 2025 | flamengo | samuel-lino -> 222 pontos
✅ Ano 2025 | flamengo | luiz-araujo -> 645 pontos
✅ Ano 2025 | flamengo | ayrton-lucas -> 336 pontos
✅ Ano 2025 | flamengo | erick-pulgar -> 565 pontos
✅ Ano 2025 | flamengo | alex-sandro -> 777 pontos
✅ Ano 2025 | flamengo | gonzalo-plata -> 488 pontos
✅ Ano 2025 | flamengo | jorginho -> 637 pontos
✅ Ano 2025 | flamengo | danilo -> 503 pontos
✅ Ano 2025 | flamengo | pedro -> 262 pontos
✅ Ano 2025 | flamengo | guillermo-varela -> 631 pontos
✅ Ano 2025 | flamengo | nicolas-de-la-cruz -> 615 pontos
✅ Ano 2025 | flamengo | michael -> 256 pontos
✅ Ano 2025 | flamengo | wallace -> 164 pontos
✅ Ano 2025 | flamengo | allan -> 492 pontos
✅ 